In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import pickle
import random
data_path = '../results/cluster_files/test_cluster2.pkl'

In [2]:
data = pickle.load(open(data_path,'rb'))
train_data = data['train']
epochs = sorted([x for x in train_data.keys()])
curr_epoch = 100
class_ids = sorted([x for x in train_data[curr_epoch].keys()])

In [3]:
train_data[curr_epoch][0][0].shape, train_data[curr_epoch][0][1].shape

((500, 640), (500, 65))

In [4]:
def stack_data(in_dict):
    class_ids = sorted(in_dict.keys())
    class_idx = {x:ctr for ctr,x in enumerate(class_ids)}
    num_classes = len(class_ids) 

    data = [in_dict[x][0] for x in class_ids]
    labels = []
    for x in class_ids:
        num_samps, num_classes_b = in_dict[x][1].shape
        assert num_classes == num_classes_b
        label_array = np.zeros((num_samps, num_classes_b))
        curr_hot = class_idx[x]
        label_array[:,curr_hot] = 1
        labels.append(label_array)

    data = np.vstack(data)
    labels = np.vstack(labels)

    return data, labels

In [5]:
data, labels = stack_data(train_data[curr_epoch])

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.shuffle(buffer_size=1024).batch(64)
data.shape, labels.shape

((32500, 640), (32500, 65))

In [7]:
reg = tf.keras.regularizers.l2(l=0.01)

In [8]:
inputs = keras.Input(shape=(640,), name="LastHidden")
outputs = tf.keras.layers.Dense(65, activation='softmax',kernel_regularizer=reg, name="Output")(inputs)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LastHidden (InputLayer)      [(None, 640)]             0         
_________________________________________________________________
Output (Dense)               (None, 65)                41665     
Total params: 41,665
Trainable params: 41,665
Non-trainable params: 0
_________________________________________________________________


In [9]:
loss_fnc = 'categorical_crossentropy'
opt = tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)

In [12]:
model.compile(
    optimizer=opt,
    loss=loss_fnc,
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)]
)

In [13]:
history = model.fit(dataset, epochs=20)

Epoch 1/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7926 - categorical_accuracy: 0.9421
Epoch 2/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7780 - categorical_accuracy: 0.9427
Epoch 3/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7669 - categorical_accuracy: 0.9424
Epoch 4/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7543 - categorical_accuracy: 0.9441
Epoch 5/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7443 - categorical_accuracy: 0.9426
Epoch 6/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7357 - categorical_accuracy: 0.9439
Epoch 7/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7270 - categorical_accuracy: 0.9430
Epoch 8/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7196 - categorical_accuracy: 0.9446
Epoch 9/20
508/508 [==============================] - 1s 3ms/step - loss: 0.7132 - categorical_accuracy: